21.	There are 1111 ways in which five 6-sided dice (sides numbered 1 to 6) can be rolled so that the top three sum to 15. Some examples are:
D1,D2,D3,D4,D5 = 4,3,6,3,5

D1,D2,D3,D4,D5 = 4,3,3,5,6

D1,D2,D3,D4,D5 = 3,3,3,6,6

D1,D2,D3,D4,D5 = 6,6,3,3,3

In how many ways can twenty 12-sided dice (sides numbered 1 to 12) be rolled so that the top ten sum to 70?


In [6]:
import itertools

DICE_COUNT = 20
FACE_COUNT = 12
TOP_DICE_COUNT = 10
TOP_DICE_SUM = 70

factorial = [1] * (DICE_COUNT + 1)

def init_factorial():
    for i in range(1, DICE_COUNT + 1):
        factorial[i] = factorial[i - 1] * i

def get_combination_count(combination):
    freqs = [0] * FACE_COUNT
    for i in combination:
        freqs[i - 1] += 1
    n, d = factorial[DICE_COUNT], 1
    for i in freqs:
        d *= factorial[i]
    return int(n/d)
    
def main():
    init_factorial()
    ways = 0
    for i in itertools.combinations_with_replacement(range(1, FACE_COUNT + 1), DICE_COUNT):
        if sum(i[DICE_COUNT - TOP_DICE_COUNT::]) == TOP_DICE_SUM:
            ways += get_combination_count(i)
    print(ways, "Can twenty 12-sided dice (sides numbered 1 to 12) be rolled so that the top ten sum to 70.")

print("Answer :")
main()

Answer :
7448717393364181966 Can twenty 12-sided dice (sides numbered 1 to 12) be rolled so that the top ten sum to 70.


23.	A triplet of positive integers (a,b,c) is called a Cardano Triplet if it satisfies the condition: a+bc√−−−−−−−√3+a−bc√−−−−−−−√3=1 For example, (2,1,5) is a Cardano Triplet. There exist 149 Cardano Triplets for which a+b+c ≤ 1000. Find how many Cardano Triplets exist such that a+b+c ≤ 110,000,000.

In [7]:
import itertools

class PrimeTable():
    def __init__(self, bound):
        self.factorization = [{} for _ in range(bound + 1)]
        self.special_factorization = [{} for _ in range(bound + 1)]
        self.primes = []
        self._sieve(bound)
    
    def _sieve(self, bound):
        visited = [False] * (bound + 1)
        visited[0] = visited[1] = True
        for i in range(2, bound + 1):
            if visited[i]:
                continue
            self.primes.append(i)
            for j in range(i, bound + 1, i):
                visited[j] = True
                self.factorization[j][i] = 1
            d = i**2
            while d <= bound:
                for j in range(d, bound + 1, d):
                    self.factorization[j][i] += 1
                d *= i
    
            # Sieve on (8k+5)-type number
            if i == 2 or 8 * bound + 5 < i**2:
                continue
            k = (-5 * self._mod_inverse(8, i**2)) % i**2
            if k > bound:
                continue
            for j in range(k, bound + 1, i**2):
                self.special_factorization[j][i] = 1
            d = i**4
            while d <= bound:
                k = (-5 * self._mod_inverse(8, d)) % d
                for j in range(k, bound + 1, d):
                    self.special_factorization[j][i] += 1
                d *= i**2
        #print(self.factorization)
        #print(self.special_factorization)
        print('Sieve completed:', len(self.primes))

    def _mod_inverse(self, a, mod):
        g, x, y = self._extended_gcd(a, mod)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % mod

    def _extended_gcd(self, a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = self._extended_gcd(b % a, a)
            return (g, x - (b//a) * y, y)

class Factorization():
    def __init__(self, bound):
        self.prime_table = PrimeTable(bound)
    
    def factorize(self, n):
        return self.prime_table.factorization[n]
    
    def get_all_divisor(self, factorization):
        unpacking = [[p**e for e in range(factorization[p] + 1)] for p in factorization]
        return [self._product(divisor) for divisor in itertools.product(*unpacking)]

    def get_problem251(self, k):
        first_part = self.factorize(k + 1) # (k + 1)^2
        second_part = self._get_square_divisor(k) # (8k + 5)
        return self._merge_two_factorization(first_part, second_part)

    def _get_square_divisor(self, k):
        return self.prime_table.special_factorization[k]

    def _merge_two_factorization(self, x, y):
        for p in y:
            if p not in x:
                x[p] = 0
            x[p] += y[p]
        return x

    def _product(self, list):
        result = 1
        for number in list:
            result *= number
        return result

class Problem():
    def __init__(self):
        self.factorization = None

    def solve(self):
        #self.get_count(10**6)
        self.get_count(110000000)

    def get_count(self, bound):
        max_k = (bound - 2)//3
        self.factorization = Factorization(max_k + 1)

        count = 0
        for k in range((bound - 2)//3 + 1):
            a = 3*k + 2
            x = (k + 1)**2 * (8 * k + 5) # where x = b^2 c
            if 4 * (bound - a)**3 < 27 * x:
                break
            x_good_factorization = self.factorization.get_problem251(k)
            for b in self.factorization.get_all_divisor(x_good_factorization):
                c = x//b**2
                if a + b + c <= bound:
                    count += 1
                    if count % 1000 == 0:
                        print(count, "=>", a, b, c)
        print(count)
        return count

def main():
    problem = Problem()
    problem.solve()

print("Answer :")
main()

Answer :
Sieve completed: 2242586
1000 => 497 1 36511700
2000 => 911 8 3507476
3000 => 1334 5 28174997
4000 => 1769 5 65679508
5000 => 2222 65 770013
6000 => 2651 221 113104
7000 => 3107 518 33140
8000 => 3554 3555 1053
9000 => 4031 24 33708864
10000 => 4502 79 4333805
11000 => 4979 20 91465253
12000 => 5459 65 11412688
13000 => 5957 662 142965
14000 => 6431 268 1097536
15000 => 6911 72 18871296
16000 => 7409 247 1975700
17000 => 7883 126 9144564
18000 => 8399 400 1097453
19000 => 8909 45 103485492
20000 => 9395 58 73054548
21000 => 9911 1652 105716
22000 => 10424 139 17373125
23000 => 10910 3637 29093
24000 => 11435 953 487888
25000 => 11951 83 73426176
26000 => 12479 130 34075648
27000 => 13007 271 8879360
28000 => 13517 4506 36045
29000 => 14039 4680 37437
30000 => 14561 129 54976404
31000 => 15095 296 11633117
32000 => 15644 5215 41717
33000 => 16163 8082 19156
34000 => 16697 253 21550100
35000 => 17207 956 1651860
36000 => 17759 2960 189428
37000 => 18311 218 38281936
38000 => 188

289000 => 180167 15014 7687120
290000 => 180851 30142 1929076
291000 => 181556 20173 4357341
292000 => 182267 91134 216020
293000 => 182984 5545 59042797
294000 => 183671 183672 54421
295000 => 184418 20491 4426029
296000 => 185129 20570 4443093
297000 => 185849 43070 1025325
298000 => 186455 7769 31821632
299000 => 187163 15597 7985616
300000 => 187904 62635 501077
301000 => 188636 62879 503029
302000 => 189341 31557 2019636
303000 => 190046 5759 61321469
304000 => 190766 63589 508709
305000 => 191483 10638 18382356
306000 => 192179 16015 8199632
307000 => 192932 4947 86947965
308000 => 193634 14895 9696037
309000 => 194354 21595 4664493
310000 => 195047 65016 520125
311000 => 195803 5439 75188304
312000 => 196484 1113415 1813
313000 => 197219 4844 96865325
314000 => 197939 6598 52783700
315000 => 198593 66198 529581
316000 => 199331 16611 8504784
317000 => 200039 33340 2133748
318000 => 200762 66921 535365
319000 => 201497 4730 108346213
320000 => 202217 33703 2156980
321000 => 20294

578000 => 397784 26519 26518925
579000 => 398609 93009 2169300
580000 => 399383 33282 17040336
581000 => 400199 66700 4268788
582000 => 400961 55034 6306269
583000 => 401759 66960 4285428
584000 => 402542 44727 9661005
585000 => 403337 67223 4302260
586000 => 404129 134710 1077677
587000 => 404930 134977 1079813
588000 => 405713 135238 1081901
589000 => 406421 45158 9754101
590000 => 407216 135739 1085909
591000 => 408029 68005 4352308
592000 => 408806 19467 53417301
593000 => 409535 17064 69893952
594000 => 410345 136782 1094253
595000 => 411155 68526 4385652
596000 => 411989 137330 1098637
597000 => 412820 40043637 13
598000 => 413630 45959 9927117
599000 => 414410 1519507 9133
600000 => 415217 138406 1107245
601000 => 415973 138658 1109261
602000 => 416807 69468 4445940
603000 => 417572 695955 44541
604000 => 418382 59769 6074285
605000 => 419180 19961 54772837
606000 => 420026 140009 1120069
607000 => 420857 140286 1122285
608000 => 421649 23425 40478292
609000 => 422486 46943 1013

844000 => 616496 205499 1643989
845000 => 617345 1440474 33597
846000 => 618176 206059 1648469
847000 => 619013 206338 1650701
848000 => 619874 206625 1652997
849000 => 620720 206907 1655253
850000 => 621572 1035955 66301
851000 => 622373 207458 1659661
852000 => 623237 29678 81436285
853000 => 624077 69342 14977845
854000 => 624941 150449 3194964
855000 => 625796 208599 1668789
856000 => 626651 50882 28162836
857000 => 627479 209160 1673277
858000 => 628271 34904 60314004
859000 => 629105 209702 1677613
860000 => 629915 29996 82308877
861000 => 630773 105129 6728244
862000 => 631616 30077 82531141
863000 => 632459 42164 42163925
864000 => 633266 211089 1688709
865000 => 634094 30195 82854933
866000 => 634922 15873075 301
867000 => 635795 30276 83077197
868000 => 636647 1061080 67909
869000 => 637487 39843 48354560
870000 => 638339 212780 1702237
871000 => 639161 213054 1704429
872000 => 639998 71111 15359949
873000 => 640862 213621 1708965
874000 => 641741 106957 6845236
875000 => 642

1104000 => 843410 281137 2249093
1105000 => 844334 16042365 693
1106000 => 845282 281761 2254085
1107000 => 846215 70518 36105168
1108000 => 847055 282352 2258813
1109000 => 847919 282640 2261117
1110000 => 848831 282944 2263549
1111000 => 849761 94418 20394261
1112000 => 850697 708915 362964
1113000 => 851624 783495 298125
1114000 => 852542 94727 20461005
1115000 => 853421 284474 2275789
1116000 => 854342 284781 2278245
1117000 => 855245 142541 9122612
1118000 => 856178 285393 2283141
1119000 => 857096 95233 20570301
1120000 => 858005 286002 2288013
1121000 => 858908 286303 2290421
1122000 => 859799 286600 2292797
1123000 => 860714 57381 57380925
1124000 => 861647 718040 367636
1125000 => 862559 287520 2300157
1126000 => 863477 863478 255845
1127000 => 864431 72036 36882384
1128000 => 865304 57687 57686925
1129000 => 866222 288741 2309925
1130000 => 867149 48175 83246292
1131000 => 868091 144682 9259636
1132000 => 869024 289675 2317397
1133000 => 869948 289983 2319861
1134000 => 87087

...

17970000 => 36101969 42118965 7858932
17971000 => 36110849 42851542 7598125
17972000 => 36119525 19441946 36937957
17973000 => 36128135 42149492 7864628
17974000 => 36137087 18068544 42829140
17975000 => 36145835 36145836 10709877
17976000 => 36154439 27718404 18225300
17977000 => 36163097 20090610 34716573
17978000 => 36171701 36171702 10717541
17979000 => 36180614 16884287 49225325
17980000 => 36189278 36189279 10722749
17981000 => 36198107 42231126 7879860
17982000 => 36206972 20114985 34758693
17983000 => 36215792 36215793 10730605
17984000 => 36223865 18111933 42931988
17985000 => 36232748 36232749 10735629
17986000 => 36241631 36241632 10738261
17987000 => 36250496 44306163 7190181
17988000 => 36259208 44316811 7191909
17989000 => 36268019 68506260 3011877
17990000 => 36276866 36276867 10748701
17991000 => 36285647 30238040 15481876
17992000 => 36294419 39318955 9163088
17993000 => 36302897 60504830 3872309
17994000 => 36311597 20173110 34859133
17995000 => 36320447 60534080 3874

18182000 => 37973897 31644915 16202196
18183000 => 37982969 18991485 45016852
18184000 => 37991654 17729439 51689325
18185000 => 38000519 20900286 37221200
18186000 => 38009483 38009484 11262069
18187000 => 38018555 29569988 18621333
18188000 => 38027735 19013868 45069908
18189000 => 38036297 63393830 4057205
18190000 => 38045447 57068172 5010100
18191000 => 38054519 28087860 20697012
18192000 => 38063132 38063133 11277965
18193000 => 38072285 38072286 11280677
18194000 => 38081105 23271787 30213108
18195000 => 38090051 38090052 11285941
18196000 => 38099072 63498455 4063901
18197000 => 38108087 38108088 11291285
18198000 => 38117147 31764290 16263316
18199000 => 38126177 19063089 45186580
18200000 => 38135303 19067652 45197396
18201000 => 38144402 38144403 11302045
18202000 => 38153522 63589205 4069709
18203000 => 38162357 19081179 45229460
18204000 => 38171447 31809540 16286484
18205000 => 38180549 17817590 51946325
18206000 => 38189303 28641978 20116176
18207000 => 38198267 19099134

18394000 => 39941657 39941658 11834565
18395000 => 39952340 31074043 19568493
18396000 => 39962897 22201610 38364381
18397000 => 39973463 39973464 11843989
18398000 => 39983939 39983940 11847093
18399000 => 39994307 19997154 47400660
18400000 => 40004789 45338762 9228325
18401000 => 40015259 40015260 11856373
18402000 => 40025897 27290385 25511156
18403000 => 40036265 40036266 11862597
18404000 => 40046903 40046904 11865749
18405000 => 40057676 40057677 11868941
18406000 => 40067804 29383057 22075925
18407000 => 40078397 33398665 17100116
18408000 => 40089185 40089186 11878277
18409000 => 40099835 36280804 14514437
18410000 => 40110083 40110084 11884469
18411000 => 40120748 57952193 5697621
18412000 => 40131062 40131063 11890685
18413000 => 40141781 40141782 11893861
18414000 => 40152257 60228387 5287540
18415000 => 40162649 50872690 7416925
18416000 => 40173344 40173345 11903213
18417000 => 40183997 33486665 17145172
18418000 => 40194323 20097162 47637716
18419000 => 40205015 20102508

18605000 => 42510599 43573365 11988800
18606000 => 42524423 31893318 22399696
18607000 => 42538436 42538437 12603981
18608000 => 42552476 42552477 12608141
18609000 => 42566774 48242345 9819325
18610000 => 42580691 42580692 12616501
18611000 => 42594542 42594543 12620605
18612000 => 42608924 33140275 20869677
18613000 => 42623027 42623028 12629045
18614000 => 42637067 42637068 12633205
18615000 => 42651134 42651135 12637373
18616000 => 42665174 42665175 12641533
18617000 => 42678947 35565790 18209684
18618000 => 42693047 59295900 6557652
18619000 => 42707105 42707106 12653957
18620000 => 42721172 42721173 12658125
18621000 => 42735347 27233310 31180788
18622000 => 42749198 42749199 12666429
18623000 => 42763022 42763023 12670525
18624000 => 42777224 42777225 12674733
18625000 => 42791318 42791319 12678909
18626000 => 42805247 35671040 18263572
18627000 => 42819248 24653507 38272421
18628000 => 42833447 23796360 41120109
18629000 => 42847748 42847749 12695629
18630000 => 42861815 589349

18816000 => 45699974 45699975 13540733
18817000 => 45716507 33017478 25969140
18818000 => 45733409 35570430 22400037
18819000 => 45749897 45749898 13555525
18820000 => 45766799 45766800 13560533
18821000 => 45783431 45783432 13565461
18822000 => 45800711 45800712 13570581
18823000 => 45817559 45817560 13575573
18824000 => 45834596 45834597 13580621
18825000 => 45851201 45851202 13585541
18826000 => 45867941 45867942 13590501
18827000 => 45884870 45884871 13595517
18828000 => 45901664 45901665 13600493
18829000 => 45918458 45918459 13605469
18830000 => 45935171 45935172 13610421
18831000 => 45952235 45952236 13615477
18832000 => 45969191 45969192 13620501
18833000 => 45986147 45986148 13625525
18834000 => 46002860 35780003 22532013
18835000 => 46019789 46019790 13635493
18836000 => 46036799 46036800 13640533
18837000 => 46053539 46053540 13645493
18838000 => 46070360 46070361 13650477
18839000 => 46087640 46087641 13655597
18840000 => 46104623 46104624 13660629
18841000 => 46121447 3843

24.	The divisors of 12 are: 1,2,3,4,6 and 12. The largest divisor of 12 that does not exceed the square root of 12 is 3. We shall call the largest divisor of an integer n that does not exceed the square root of n the pseudo square root (PSR) of n. It can be seen that PSR(3102)=47. Let p be the product of the primes below 190. Find PSR(p) mod 1016.

In [11]:
import math

class PrimeTable():
    def __init__(self, bound):
        self.bound = bound
        self.primes = []
        self._sieve()
    
    def _sieve(self):
        visited = [False] * (self.bound + 1)
        visited[0] = visited[1] = True
        for i in range(2, self.bound + 1):
            if not visited[i]:
                self.primes.append(i)
            for j in range(i + i, self.bound + 1, i):
                visited[j] = True
        print('Prime count:', len(self.primes))

class Factorization():
    def __init__(self):
        self.prime_table = PrimeTable(190)
    
    def factorize(self, n):
        d = n
        rv = []
        for i in range(len(self.prime_table.primes)):
            p = self.prime_table.primes[i]
            if d == 1 or p > d:
                break
            count = 0
            while d % p == 0:
                d = d//p
                count += 1
            if count > 0:
                rv += [p] * count
        if d > 1:
            rv.append(d)
        return rv

class Problem():
    def __init__(self):
        self.factorization = Factorization()
        self.primes = self.factorization.prime_table.primes
        self.lower_half_array = None
        self.upper_half_array = None

    def solve(self):
        print(self.pseudo_square_root(self.factorization.factorize(12)))
        print(self.pseudo_square_root(self.factorization.factorize(3102)))
        print(self.pseudo_square_root(self.primes))

    def pseudo_square_root(self, prime_factors):
        upper_bound = self.get_upper_bound(prime_factors)
        half_len = len(prime_factors)//2
        self.lower_half_array = self.populate(prime_factors[:half_len], upper_bound)
        self.upper_half_array = self.populate(prime_factors[half_len:], upper_bound)
        best_so_far = 0
        for n in self.lower_half_array:
            x = self.binary_search(n, upper_bound)
            if x > best_so_far:
                best_so_far = x
                print('Current best:', n, x, best_so_far)
        return best_so_far % 10**16

    def binary_search(self, n, upper_bound):
        L = 0
        U = len(self.upper_half_array) - 1
        while L <= U:
            M = (L + U)//2
            if self.upper_half_array[M] * n < upper_bound:
                L = M + 1
            else:
                U = M - 1
        return self.upper_half_array[U] * n

    def get_upper_bound(self, n):
        product = 1
        for i in n:
            product *= i
        return int(math.sqrt(product)) + 1

    def populate(self, prime_factors, upper_bound):
        rv = [1]
        for d in prime_factors:
            next = [d * x for x in rv if d * x < upper_bound]
            rv += next
        rv.sort()
        return rv

def main():
    problem = Problem()
    problem.solve()
    
print("Answer :")
main()

Answer :
Prime count: 42
Current best: 1 3 3
3
Current best: 1 47 47
47
Current best: 1 2248291997912258509664470404609009559 2248291997912258509664470404609009559
Current best: 2 2323110152219471434378288920861013498 2323110152219471434378288920861013498
Current best: 3 2323187337087310475890494990663686799 2323187337087310475890494990663686799
Current best: 247 2323216826486856640868123075940942287 2323216826486856640868123075940942287
Current best: 473 2323217706934997506391362393294770451 2323217706934997506391362393294770451
Current best: 667 2323218881173926278494148853633342943 2323218881173926278494148853633342943
Current best: 75790 2323218934679017229511810488219357330 2323218934679017229511810488219357330
Current best: 83549 2323218946350798495292922250780318257 2323218946350798495292922250780318257
Current best: 99897 2323218948224643767029806368186417133 2323218948224643767029806368186417133
Current best: 8490910 2323218950509656390291802533969867770 2323218950509656390291